# Doppler Ultrasound Signal Analysis

This notebook demonstrates processing and visualization of Doppler Ultrasound (DUS) signals for fetal heart rate analysis.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os
import scipy.io.wavfile as wavfile
import pywt
from IPython.display import Audio
import sys

# Add project root to path to import project modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))
from AutoFHR.src.utils.visualization import plot_scalogram, play_audio
from AutoFHR.src.utils.signal_processing import normalize, signal_resample, butter_bandpass_filter, DUS_filtering, create_scalogram

# Set some plotting defaults
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

## Loading and Playing WAV Files

Load the sample WAV files and play them.

In [ ]:
# List all WAV files and output of the model if exists in the data directory
data_dir = "data"
wav_files = [f for f in os.listdir(data_dir) if f.endswith('.wav')]
npy_files = [f for f in os.listdir(data_dir) if f.endswith('.npy')]
print(f"Found {len(wav_files)} WAV files:")
for file in wav_files:
    print(f"- {file}")
print(f"\nFound {len(npy_files)} npy files:")
for file in npy_files:
    print(f"- {file}")

In [6]:
# Function to load WAV file and return rate and data
def load_wav_file(file_path):
    """Load a WAV file and return sample rate and data"""
    try:
        sample_rate, data = wavfile.read(file_path)
        print(f"Loaded {file_path}")
        print(f"Sample rate: {sample_rate} Hz")
        print(f"Data shape: {data.shape}")
        print(f"Duration: {data.shape[0]/sample_rate:.2f} seconds")
        return sample_rate, data
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None, None

In [ ]:
# Load and play the first WAV file
if len(wav_files) > 0:
    file_path = os.path.join(data_dir, wav_files[-1])
    fs, dus_data = load_wav_file(file_path)
    
    # Convert to mono if stereo
    if len(dus_data.shape) > 1 and dus_data.shape[1] > 1:
        dus_data = dus_data[:, 0] 
    
    # Normalize data for better playback
    dus_data_norm = normalize(dus_data)
    
    # Play the audio
    print("\nPlaying original audio:")
    play_audio(file_path)

## Signal Processing

Let's process the DUS signal using basic filtering and normalization.

In [ ]:
# Apply filtering to the DUS signal
if 'dus_data' in locals() and dus_data is not None:
    fs_t = 4000
    dus_filtered = normalize(DUS_filtering(signal_resample(dus_data,fs,fs_t), fs_t))
    
    plt.figure(figsize=(14, 8))
    
    t = np.arange(len(dus_filtered)) / fs_t
    
    plt.subplot(2, 1, 1)
    plt.plot(t[:int(fs_t*3.75)], dus_filtered[:int(fs_t*3.75)], 'b-')
    plt.title('Original DUS Signal')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.plot(t[:int(fs_t*3.75)], dus_filtered[:int(fs_t*3.75)], 'r-')
    plt.title('Filtered DUS Signal')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Play the filtered audio
    print("\nPlaying filtered audio:")
    Audio(data=dus_filtered.astype(np.int16), rate=fs)

## Creating and Visualizing Scalograms

Create and plot scalograms of the DUS signal to visualize frequency components over time.

In [ ]:
# Create scalogram for visualization
if 'dus_filtered' in locals() and dus_filtered is not None:
    signal_segment = dus_filtered
    time_bins = 1000 
    freq_bins = 40    
    
    # Create the scalogram
    scalogram = create_scalogram(signal_segment, fs_t, time_bins, freq_bins)

    scales = np.arange(1, freq_bins + 1)
    fc = pywt.central_frequency('morl')
    frequencies = fc * fs_t / scales
    
    t = np.linspace(0, 3.75, time_bins)
    
    # Plot the scalogram using our custom function
    fig, axes = plot_scalogram(
        scalogram=scalogram,
        times=t,
        frequencies=frequencies,
        dus_signal=signal_segment,
        estimated = np.load(os.path.join(data_dir,'output_3.npy')),
        fs=fs_t,
        figsize=(12, 6),
        title=f"Scalogram of DUS Signal from {wav_files[0]}",
        cmap='plasma'
    )
    
    plt.show()

## Batch Processing of All WAV Files

Let's process all the WAV files in the data directory, create scalograms, and save the results.

In [15]:
def process_wav_file(file_path, fs_t=4000, save_dir="results"):
    """Process a WAV file, create scalogram, and save results"""
    # Create save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)
    
    # Get filename without extension
    filename = os.path.splitext(os.path.basename(file_path))[0]
    
    # Load WAV file
    fs, dus_data = load_wav_file(file_path)
    
    if fs is None or dus_data is None:
        return None
    
    # Convert to mono if stereo
    if len(dus_data.shape) > 1 and dus_data.shape[1] > 1:
        dus_data = dus_data[:, 0]  
    
    dus_filtered = normalize(DUS_filtering(signal_resample(dus_data,fs,fs_t), fs_t))
    
    segment_length = min(fs_t * 3.75, len(dus_filtered)) 
    signal_segment = dus_filtered
    
    # Create scalogram
    time_bins = 1000
    freq_bins = 40
    scalogram = create_scalogram(signal_segment, fs_t, time_bins, freq_bins)
    
    # Calculate frequency range
    scales = np.arange(1, freq_bins + 1)
    fc = pywt.central_frequency('morl')
    frequencies = fc * fs_t / scales
    
    t = np.linspace(0, len(signal_segment)/fs_t, time_bins) 
    
    # Plot and save the scalogram
    fig, axes = plot_scalogram(
        scalogram=scalogram,
        times=t,
        frequencies=frequencies,
        dus_signal=signal_segment,
        fs=fs_t,
        figsize=(12, 6),
        title=f"Scalogram of DUS Signal from {filename}",
        save_path=os.path.join(save_dir, f"{filename}_scalogram.png"),
        cmap='plasma'
    )
    
    plt.close(fig)  # Close the figure to avoid displaying
    
    # Save the processed data
    np.savez(
        os.path.join(save_dir, f"{filename}_processed.npz"),
        dus_original=dus_data,
        dus_filtered=dus_filtered,
        scalogram=scalogram,
        frequencies=frequencies,
        fs=fs_t
    )
    
    return {
        'filename': filename,
        'fs': fs_t,
        'duration': len(dus_data) / fs,
        'scalogram_shape': scalogram.shape
    }

In [16]:
# Process all WAV files
results = []
for wav_file in wav_files:
    file_path = os.path.join(data_dir, wav_file)
    print(f"\nProcessing {wav_file}...")
    result = process_wav_file(file_path)
    if result is not None:
        results.append(result)
        print(f"Processed {wav_file} successfully.")

# Display summary
print("\nProcessing Summary:")
for result in results:
    print(f"- {result['filename']}: Duration={result['duration']:.2f}s, FS={result['fs']}Hz, Scalogram Shape={result['scalogram_shape']}")